In [ ]:
#import modules
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [ ]:
#LSAT dataset

import os
import tempfile
import six.moves.urllib as urllib
import pprint
_DATA_ROOT = tempfile.mkdtemp(prefix='lsat-data')
_DATA_PATH = 'https://storage.googleapis.com/lawschool_dataset/bar_pass_prediction.csv'
_DATA_FILEPATH = os.path.join(_DATA_ROOT, 'bar_pass_prediction.csv')

data = urllib.request.urlopen(_DATA_PATH)

_LSAT_DF = pd.read_csv(data)

# To simpliy the case study, we will only use the columns that will be used for
# our model.
_COLUMN_NAMES = [
  'gender',
  'lsat',
  'pass_bar',
  'race1',
  'ugpa',
]

_LSAT_DF.dropna()
_LSAT_DF['gender'] = _LSAT_DF['gender'].astype(str)
_LSAT_DF['race1'] = _LSAT_DF['race1'].astype(str)
_LSAT_DF = _LSAT_DF[_COLUMN_NAMES]

_LSAT_DF= _LSAT_DF=_LSAT_DF[_LSAT_DF['gender']!='nan']
_LSAT_DF=_LSAT_DF[_LSAT_DF['race1']!='other']
_LSAT_DF=_LSAT_DF[_LSAT_DF['race1']!='nan']

_LSAT_DF['race1'][_LSAT_DF['race1']=='white']=1
_LSAT_DF['race1'][_LSAT_DF['race1']=='hisp']=2
_LSAT_DF['race1'][_LSAT_DF['race1']=='asian']=3
_LSAT_DF['race1'][_LSAT_DF['race1']=='black']=4

_LSAT_DF['gender'][_LSAT_DF['gender']=='male']=1
_LSAT_DF['gender'][_LSAT_DF['gender']=='female']=0

tr=_LSAT_DF[['gender','lsat','race1','ugpa']]
tar=_LSAT_DF['pass_bar'].values


tar=tar.astype('int')
tr, cat_names=to_binary(tr,['race1'],[4])
names=list(tr.columns)
tr=tr.values.astype(float)

In [ ]:
#modify dataset turning categorical to binary variables

def to_binary(dataset, categorical_vars, num_of_categories):
    new_vars=[]
    for var,cat in zip(categorical_vars, num_of_categories):
        temp=[]
        for c in range(cat):
            dataset[var+'_'+str(c+1)]=0
            dataset.loc[dataset[var] == c+1, var+'_'+str(c+1)] = 1
            temp.append(var+'_'+str(c+1))
        new_vars.append(temp)
        dataset = dataset.drop(var, 1)
    return dataset, new_vars

In [ ]:
#train random forests and export texts; for example
rf = RandomForestClassifier(random_state=0, n_estimators=5, max_depth= 100)
rf = rf.fit(tr, tar)

textdt=[]

for dt in rf.estimators_:
    textdt.append(export_text(dt, feature_names=names,max_depth=1000000))

In [ ]:
#function for predicting based onmajority voting
def prediction(rf,datapoint):
    ones=0
    zeros=0
    for dt in rf.estimators_:
        if (dt.predict(datapoint.reshape(1, -1))==1):
            ones=ones+1
        elif (dt.predict(datapoint.reshape(1, -1))==0):
            zeros=zeros+1
    if (ones<zeros):
        return 0
    elif (ones>=zeros):
        return 1

In [ ]:
import string
variablenames=list(string.ascii_lowercase)[:len(textdt)]
treecount=0
treerules=[]
tree_feature_to_boolean=[]
tree_variable_to_boolean_and_feature=[]
tree_polynomials=[]
listofvariables=[]


boolean_rules=[]
boolean_to_tree_feature=[]
list_of_rules=[]

In [ ]:
#function for making the positive decision polynomial
def make_positive_polynomial_and_dicts(textdt, names=names): 
    #for eah tree turns exported text to list of rules
    list_of_rules=[]
    boolean_to_tree_feature=[] 
    boolean_rules=[]
    tree_polynomials=[] 
    tree_variable_to_boolean_and_feature=[]
    tree_feature_to_boolean=[]  
    i=1
    for r in textdt:
        dt=[]
        rules = r.splitlines()
        currentrule=[]
        for rule in rules:
            depth = rule.count("|")
            rule=rule.split("--- ",1)[1]
            if (depth<len(currentrule)):

                dt.append(currentrule)
                currentrule=currentrule[:depth-1]
                currentrule.append(rule)
                continue
            currentrule.append(rule)
        dt.append(currentrule)

        list_of_rules.append(dt)

    #turns dranching rules to binary variables, introducing one variable per rule
        dict={}
        dict1={}

        for rule in dt:
            for feat in rule:
                if ((">" in feat) or ("class" in feat)):
                    continue
                if (feat not in dict.values()):
                    if ([True for pr_dict in boolean_to_tree_feature if feat in pr_dict.values()]==[]):
                        dict["x"+str(i)]=feat             
                        i=i+1
        dict1=dict
        dict={v: k for k, v in dict.items()}

        tree_feature_to_boolean.append(dict) 
        boolean_to_tree_feature.append(dict1)

    #rewrites decision tree internal rules in terms of the new binary variables
        boolrules=[]
        temp=[]
        for rule in dt:
            for feat in rule:
                if ">" in feat:
                    terms=feat.split(">",1)
                    
                    val=[value for key, value in dict.items() if terms[0].strip() in key and terms[1].strip() == key.split(' ')[-1]]
                    if (val==[]):
                        val=[value for pr_dict in tree_feature_to_boolean for key, value in pr_dict.items() if terms[0].strip() in key and terms[1].strip() == key.split(' ')[-1]]
                    val=val[0]   
                    temp.append(str(val)+"=0")
                elif "<=" in feat:
                    try:
                        val=dict[feat]
                    except:
                        for pr_dict in tree_feature_to_boolean:   #mine new
                            if (feat in pr_dict.keys()):
                                val=pr_dict[feat]
                                break
                    temp.append(str(val)+"=1")
                else:
                    temp.append(feat)
            boolrules.append(temp)
            temp=[]

        boolean_rules.append(boolrules)
        #construct positive and negative polynomials (as rules)
        positiverules=[]
        negrules=[]
        for rule in boolrules:
            if (rule[-1]=="class: 1.0"):
                positiverules.append(rule[:-1])
            else:
                negrules.append(rule[:-1])
        tree_polynomials.append((positiverules,negrules)) 

    #construct dictionary that maps the original variables to the features involving them, as well as the binary variables they correspond to
        featurenames=names
        d= dict.items()
        newdict={}
        for feat in featurenames:
            keys = [(float(key.split(" ")[-1]),val) for key,val in d if feat in key]
            newdict[feat]= keys

        tree_variable_to_boolean_and_feature.append(newdict)
        

    newdict={}  
    for feat in featurenames:
        newdict[feat]=[]
        for pr_dict in tree_variable_to_boolean_and_feature:
            newdict[feat]=newdict[feat]+pr_dict[feat]

    return boolean_to_tree_feature, tree_polynomials, tree_feature_to_boolean, newdict 


In [ ]:
#import solver
from ortools.linear_solver import pywraplp
#initialize solver
solver = pywraplp.Solver.CreateSolver('SCIP')

In [ ]:
def make_variables_and_constraints(tree_feature_to_boolean, tree_polynomials, counter_outcome, solver):
    import math 
   
    deltasum=0
    listofvariables=[] 
    for tree in range(len(tree_polynomials)):

    #create the actual binary variables
        variables=[]
        for var in tree_feature_to_boolean[tree].values(): 
            listofvariables.append(solver.IntVar(0.0, 1.0, var)) 

    #construct the left hand side of the constraints, assuming we use the positive polynomial
        constrs=[]
        num=0
        for rule in tree_polynomials[tree][0]: 
            sum=0
            for feat in rule:
                
                if (feat[-1]=="0"):
                    sum=sum + 1-listofvariables[int(''.join(filter(str.isdigit, feat.split('=',1)[0])))-1] 
                else:                                     
                    sum=sum + listofvariables[int(''.join(filter(str.isdigit, feat.split('=',1)[0])))-1]  
                num=num+1                                 
            constrs.append((sum,num))
            num=0
        if(counter_outcome==0):
        #add constraints to enforce negative outcome, using the positive polynomial (so all positive rules must be zero)
            sum=0
            delta=solver.IntVar(0.0, 1.0, 'delta')
            for const in constrs:
                solver.Add(const[0] - const[1] <= delta -1)
            deltasum=deltasum+delta
        elif(counter_outcome==1):
    #alternatively, these constraints enforce positive outcome (using the new result, having the positive polynomial equal 1)

            sum=0
            for const in constrs:
                delta=solver.IntVar(0.0, 1.0, 'delta')
                solver.Add(const[0] >= delta*const[1])
                sum=sum+delta
            deltaprime=solver.IntVar(0.0, 1.0, 'Delta') 
            deltasum=deltasum + deltaprime
            solver.Add(sum==deltaprime)

    
    if (counter_outcome==0):
        solver.Add(deltasum<=int(math.ceil(len(textdt)/2))-1)
    elif(counter_outcome==1):
        solver.Add(deltasum>=int(math.ceil(len(textdt)/2)))  

    return listofvariables

In [ ]:
#cat_names is a list of lists with the new categorical names, in the form X_1, X_2...
def add_categorical_constraints(variables, cat_names, dict1, solver):

    for cat_vars in cat_names:
        sum=0
        i=0
        for var in cat_vars:
            bool_vars=[val for key,val in dict1.items() if var in key]
            for feats in bool_vars:
                index = int(''.join(filter(str.isdigit, feats[0][1]))) -1
                sum=sum + variables[index]
                i=i+1
        solver.Add(sum==i-1)

In [ ]:
#diversity constraints
#the 3 first are list of tuples: (feat, _constraint value)
#must have initialized the optimization problem and all dicts and remaining constraints. This is the last step before solving!
def diverse_counterfactual_constraints(eq_constrs, leq_constrs, gr_constrs,solver, newdict, variables):
    for constr in eq_constrs:
        feat, constr_val = constr
        feats = newdict[feat]
        for val, var in feats:
            index= int(''.join(filter(str.isdigit, var))) -1
            if (constr_val<= val):
                solver.Add(variables[index] ==1)
            else:
                solver.Add(variables[index] ==0)
                
    for constr in leq_constrs:
        feat, constr_val = constr
        feats = newdict[feat]
        for val, var in feats:
            index= int(''.join(filter(str.isdigit, var))) -1
            if (constr_val<= val):
                solver.Add(variables[index] ==1)
                
    for constr in gr_constrs:
        feat, constr_val = constr
        feats = newdict[feat]
        for val, var in feats:
            index= int(''.join(filter(str.isdigit, var))) -1
            if (constr_val > val):
                solver.Add(variables[index] ==0)

In [ ]:
#this computes the transformed data point

def make_actual_datapoint(datapoint, newdict, featurenames):
    num_of_feats=np.sum([len(value) for value in newdict.values()])
    new_datapoint=np.zeros(num_of_feats)
    new_datapoint_indices=np.zeros(num_of_feats)
    sum=0 
    mini_sum=0
    i=0
    j=0
    for feat in featurenames:
        current_tree_features=newdict[feat]
        for rules in current_tree_features:
            index= int(''.join(filter(str.isdigit, rules[1]))) -1
            new_datapoint_indices[j]=index
            cond= rules[0]
            if (datapoint[i]<= cond):
                new_datapoint[j]=1
            else:
                new_datapoint[j]=0
            j=j+1
        i=i+1
    return new_datapoint, new_datapoint_indices

In [ ]:
#computes standard deviation
def make_coefficients(tr, new_datapoint, featurenames, newdict):
    sum=0
    i=0
    mini_sum=0
    binary=np.zeros(len(new_datapoint))
    j=0
    for feat in featurenames:
        current_tree_features=newdict[feat]
        for rules in current_tree_features:
            index= int(''.join(filter(str.isdigit, rules[1]))) -1
            cond= rules[0]
            masked_data= tr[:,i]   # dataset[:,i]
            n=len(masked_data)
            masked_data=masked_data[masked_data<=cond]
            p=len(masked_data)
            p=p/n
            variance=p*(1-p)
            binary[j]=np.sqrt(variance)
            j=j+1
        i=i+1
    return binary

In [ ]:
#construct the objective function
def make_objective(actualdatapoint, coeff, listofvariables, new_datapoint_indices, solver):
    obj=0
    index=0
    for feat in actualdatapoint:
        if feat==0:
            obj=obj+coeff[index]*listofvariables[int(new_datapoint_indices[index])]
        else:
            obj=obj+coeff[index]*(1-listofvariables[int(new_datapoint_indices[index])])
        index=index+1
    solver.Minimize(obj)

In [ ]:
def written_counterfactual(listofvariables, new_datapoint, new_datapoint_indices, boolean_to_tree_feature):
    dict1 = {}
    for dictionary in boolean_to_tree_feature:
        dict1.update(dictionary)

    solution=[]
    for var in listofvariables:
            solution.append(var.solution_value())

    counter_changes=[]
    counter_indices=np.where(new_datapoint[np.argsort(np.array(list(map(int,new_datapoint_indices))))]!=solution)[0]

    for counter_index in counter_indices:
        if solution[counter_index]==1:
            counter_changes.append(dict1['x'+str(counter_index+1)])
        elif solution[counter_index]==0:
            counter_changes.append(dict1['x'+str(counter_index+1)].replace('<=','>'))

    ls= list(map(lambda x: x.split('<=')+['<='] if '<=' in x  else x.split('>')+['>'], counter_changes))
    from collections import defaultdict
    d = defaultdict(list)

    for k, *v in ls:
        d[k].append(v)

    bn=list(d.items())
    grouped=[]
    for p, r in bn:
        greater=[]
        lower=[]
        for k, v in r:
            if (v=='<='):
                lower.append(k)
            elif (v=='>'):
                greater.append(k)
        lower=['<=']+lower
        greater=['>']+greater
        grouped.append([p,lower,greater])
    written_counterfactuals=[]
    for feature,lower,greater in grouped:
        s=feature
        if(len(lower)>1):
            u_bound=min(map(float,lower[1:]))
            s=s+ '<= '+ str(u_bound)
        if(len(greater)>1):
            l_bound=max(map(float,greater[1:]))
            s= str(l_bound) + ' < ' +s
        written_counterfactuals.append(s)
    return written_counterfactuals

In [ ]:
import random
K=5
random.seed(101)
indices = random.sample(range(len(tr)), K)
exp_data=[tr[i] for i in sorted(indices)]
exp_tar=[prediction(rf, exp_data[i]) for i in range(len(exp_data))]

boolean_to_tree_feature, tree_polynomials, tree_feature_to_boolean, newdict = make_positive_polynomial_and_dicts(textdt)

counterfactuals=[]

for i in range(K):
    datapoint = exp_data[i]
    counter_outcome = 0 if exp_tar[i] == 1 else 1
    
    #initialize solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    variables=make_variables_and_constraints(tree_feature_to_boolean, tree_polynomials, counter_outcome, solver)
    
    if (i==0):
        diverse_counterfactual_constraints([('race1_1',1)], [], [],solver, newdict, variables)
    elif(i==1):
        diverse_counterfactual_constraints([],[], [('lsat',30)],solver, newdict, variables)
    elif(i==2):
        diverse_counterfactual_constraints([('race1_1',1)], [], [],solver, newdict, variables)
    elif(i==3):
        diverse_counterfactual_constraints([],[], [('lsat',25)],solver, newdict, variables)
    else:
        diverse_counterfactual_constraints([('race1_1',1)], [], [],solver, newdict, variables)
    
    #add_constraints(positiverules,variables, counter_outcome, solver)
    add_categorical_constraints(variables, [['race1_1','race1_2','race1_3','race1_4']], newdict, solver)
    new_datapoint, new_datapoint_indices = make_actual_datapoint(datapoint, newdict, names)
    coeff = make_coefficients(tr, new_datapoint, names, newdict)
    make_objective(new_datapoint, coeff, variables, new_datapoint_indices, solver)
    status = solver.Solve()
    cntr=written_counterfactual(variables, new_datapoint, new_datapoint_indices, boolean_to_tree_feature)
    counterfactuals.append(cntr)